In [ ]:
# Install and fix PyArrow compatibility issues
import subprocess
import sys
import importlib

def check_and_fix_pyarrow():
    """Check PyArrow version and fix compatibility issues"""
    print("🔍 Checking PyArrow compatibility...")
    
    try:
        import pyarrow as pa
        print(f"📦 PyArrow version: {pa.__version__}")
        
        # Test for the specific attribute that's causing issues
        if hasattr(pa.lib, 'PyExtensionType'):
            print("✅ PyExtensionType is available")
            return True
        else:
            print("❌ PyExtensionType not found - version issue detected")
            return False
            
    except ImportError:
        print("❌ PyArrow not installed")
        return False

def fix_pyarrow_compatibility():
    """Fix PyArrow version compatibility"""
    print("\n🔧 Fixing PyArrow compatibility...")
    
    try:
        # Uninstall and reinstall with specific compatible versions
        print("📦 Uninstalling existing PyArrow...")
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"])
        
        print("📦 Installing compatible PyArrow version...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pyarrow>=12.0.0,<15.0.0"])
        
        # Also ensure pandas compatibility
        print("📦 Updating pandas for compatibility...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pandas>=1.5.0"])
        
        print("✅ PyArrow compatibility fix complete!")
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to fix PyArrow: {e}")
        return False

def install_package(package_name, import_name=None):
    """Install a package and verify it can be imported"""
    if import_name is None:
        import_name = package_name
    
    try:
        # Try to import the package
        importlib.import_module(import_name)
        print(f"✅ {package_name} is already installed")
        return True
    except ImportError:
        print(f"📦 Installing {package_name}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"✅ {package_name} installed successfully")
            return True
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package_name}: {e}")
            return False

print("🔧 Setting up BioTrove Reptilia processing environment...")
print("=" * 60)

# First, check and fix PyArrow if needed
if not check_and_fix_pyarrow():
    print("\n🛠️ Attempting to fix PyArrow compatibility...")
    if fix_pyarrow_compatibility():
        # Re-check after fix
        if check_and_fix_pyarrow():
            print("🎉 PyArrow issue resolved!")
        else:
            print("⚠️ PyArrow issue persists - may need manual intervention")
    else:
        print("❌ Could not automatically fix PyArrow")

print("\n" + "=" * 60)

# List of required packages (with specific versions to avoid conflicts)
required_packages = [
    ("arbor-process", "arbor_process"),
    ("nest_asyncio", "nest_asyncio"),
    ("datasets>=2.14.0", "datasets"),
    ("pandas>=1.5.0", "pandas"),
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("seaborn", "seaborn"),
    ("pillow", "PIL"),
    ("requests", "requests"),
    ("tqdm", "tqdm")
]

# Install packages
all_installed = True
for package_name, import_name in required_packages:
    if not install_package(package_name, import_name):
        all_installed = False

if all_installed:
    print("\n🎉 All packages installed successfully!")
    print("📚 Ready to process BioTrove Reptilia dataset")
else:
    print("\n⚠️  Some packages failed to install")
    print("💡 You may need to install them manually")

print("\n💡 If you still get PyArrow errors, try:")
print("   pip install --force-reinstall pyarrow==14.0.2")
print("   pip install --force-reinstall pandas==2.0.3")

print("\n" + "=" * 60)

In [ ]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from PIL import Image
from io import BytesIO
from tqdm.auto import tqdm
import warnings
import json
import asyncio

# Import BioTrove official processing library
try:
    from arbor_process import *
    print("✅ BioTrove processing library (arbor_process) imported successfully!")
except ImportError:
    print("❌ BioTrove processing library not found!")
    print("💡 Please run the installation cell above first")
    print("💡 If that fails, try: pip install arbor-process")

# Set up environment
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

print("📦 Core libraries loaded successfully!")
print("🐍 Ready to work with BioTrove Reptilia dataset using official preprocessing tools")

In [ ]:
# Verify BioTrove processing tools are available
print("🔍 Verifying BioTrove processing tools...")

try:
    # Test if we can access the main BioTrove processing classes
    from arbor_process import MetadataProcessor, GenShuffledChunks, GetImages, GenImgTxtPair, load_config
    
    print("✅ MetadataProcessor - Available")
    print("✅ GenShuffledChunks - Available") 
    print("✅ GetImages - Available")
    print("✅ GenImgTxtPair - Available")
    print("✅ load_config - Available")
    print("\n🎉 All BioTrove processing tools are ready!")
    print("🦎 Proceeding with Reptilia dataset processing...")
    
except ImportError as e:
    print(f"❌ BioTrove tools not available: {e}")
    print("\n🛠️ TROUBLESHOOTING:")
    print("1. Run the installation cell above")
    print("2. Restart the kernel and run all cells from the beginning")
    print("3. If still failing, install manually:")
    print("   pip install arbor-process")
    print("\n💡 You can also use the legacy streaming method as fallback")

In [ ]:
# Download metadata files from the CORRECT Hugging Face source
print("📥 Downloading BioTrove metadata files from BGLab/BioTrove-Train...")
print("🔄 This is the correct metadata source according to the official GitHub repository")

# Create directories for metadata
os.makedirs("biotrove_metadata", exist_ok=True)
os.makedirs("biotrove_processed", exist_ok=True)

def download_biotrove_metadata():
    """Download metadata from the correct BGLab/BioTrove-Train dataset"""
    print("📡 Loading BioTrove metadata from Hugging Face...")
    
    try:
        from datasets import load_dataset
        
        # Load the correct dataset that contains the metadata
        print("🔍 Accessing BGLab/BioTrove-Train dataset...")
        dataset = load_dataset("BGLab/BioTrove-Train", streaming=True)
        
        print("✅ Successfully connected to BioTrove-Train dataset!")
        print("📊 Dataset structure:")
        print(f"   Available splits: {list(dataset.keys())}")
        
        # Let's examine the structure first
        train_data = dataset['train'] if 'train' in dataset else next(iter(dataset.values()))
        
        # Get sample to understand structure
        print("\n🔍 Examining dataset structure...")
        print("💡 Searching more extensively for Reptilia samples...")
        print("⚠️ Note: Dataset has 135M samples - this may take a few minutes")
        
        sample_count = 0
        reptile_samples = []
        all_classes_seen = set()
        check_points = [0, 50000, 100000, 500000, 1000000]  # Check different positions
        
        from tqdm.auto import tqdm
        pbar = tqdm(desc="Scanning for reptiles", unit=" samples")
        
        for item in train_data:
            sample_count += 1
            
            # Check if this is reptile data
            if isinstance(item, dict):
                class_value = item.get('class', item.get('category', ''))
                all_classes_seen.add(str(class_value))
                
                if class_value == 'Reptilia' or 'reptil' in str(class_value).lower():
                    reptile_samples.append(item)
                    if len(reptile_samples) <= 10:  # Show first few samples
                        print(f"\n   🦎 Found Reptilia sample {len(reptile_samples)}: {item.get('species', 'Unknown species')}")
                        pbar.set_description(f"Found {len(reptile_samples)} reptiles!")
            
            # Update progress
            if sample_count % 10000 == 0:
                pbar.set_description(f"Processed {sample_count:,} | Found {len(reptile_samples)} reptiles")
                pbar.update(10000)
                
                # Show what classes we've seen so far
                if sample_count % 50000 == 0:
                    print(f"\n📊 Classes seen so far ({sample_count:,} samples): {sorted(list(all_classes_seen))}")
            
            # Stop if we found enough reptiles or processed enough samples
            if len(reptile_samples) >= 1000:  # Stop after finding 1000 reptiles
                print(f"\n🎉 Found {len(reptile_samples)} reptiles! Stopping search.")
                break
            elif sample_count >= 2000000:  # Stop after 2M samples if no reptiles found
                print(f"\n⚠️ Searched {sample_count:,} samples without finding Reptilia")
                break
        
        pbar.close()
        
        if reptile_samples:
            print(f"\n🎉 Found {len(reptile_samples)} Reptilia samples in {sample_count:,} records!")
            
            # Create sample parquet files for processing
            chunk_size = 500
            saved_files = []
            
            for i in range(0, len(reptile_samples), chunk_size):
                chunk = reptile_samples[i:i+chunk_size]
                chunk_df = pd.DataFrame(chunk)
                
                chunk_filename = f"biotrove_metadata/reptilia_chunk_{i//chunk_size}.parquet"
                chunk_df.to_parquet(chunk_filename, index=False)
                saved_files.append(chunk_filename)
                
                print(f"💾 Saved chunk {i//chunk_size}: {len(chunk)} samples to {chunk_filename}")
            
            return saved_files
        else:
            print(f"⚠️ No Reptilia samples found in first {sample_count:,} records")
            print(f"📊 All classes seen: {sorted(list(all_classes_seen))}")
            
            # The dataset might have a different structure - let's save what we found
            sample_data = []
            train_data_fresh = dataset['train'] if 'train' in dataset else next(iter(dataset.values()))
            
            print("💡 Saving sample data for analysis...")
            for i, item in enumerate(train_data_fresh):
                sample_data.append(item)
                if i >= 500:  # Get 500 samples to analyze
                    break
            
            if sample_data:
                sample_df = pd.DataFrame(sample_data)
                sample_file = "biotrove_metadata/sample_structure.parquet"
                sample_df.to_parquet(sample_file, index=False)
                
                print(f"\n📊 Created sample file to analyze structure: {sample_file}")
                print(f"   📏 Shape: {sample_df.shape}")
                print(f"   🏷️ Columns: {list(sample_df.columns)}")
                
                # Show unique values in potential class columns
                for col in sample_df.columns:
                    if 'class' in col.lower() or 'category' in col.lower() or 'tax' in col.lower():
                        unique_vals = sample_df[col].unique()[:10]
                        print(f"   🔍 {col} values: {unique_vals}")
                
                return [sample_file]
            
        return []
        
    except Exception as e:
        print(f"❌ Error accessing BioTrove-Train dataset: {e}")
        print("\n🛠️ TROUBLESHOOTING:")
        print("1. Check internet connection")
        print("2. Verify dataset exists: https://huggingface.co/datasets/BGLab/BioTrove-Train")
        print("3. Try alternative download methods")
        return []

def try_direct_download_from_hf():
    """Try downloading files directly from HuggingFace file browser"""
    print("\n🔄 Trying direct file download from HuggingFace...")
    
    base_url = "https://huggingface.co/datasets/BGLab/BioTrove-Train/resolve/main/"
    
    # Common file patterns to try
    file_patterns = [
        "data.parquet",
        "train.parquet", 
        "metadata.parquet",
        "arboretum.parquet",
        "dataset.parquet"
    ]
    
    downloaded_files = []
    
    for filename in file_patterns:
        url = base_url + filename
        filepath = f"biotrove_metadata/{filename}"
        
        try:
            print(f"⬇️  Trying to download {filename}...")
            
            response = requests.get(url, timeout=30)
            if response.status_code == 200:
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                # Verify it's a valid parquet file
                try:
                    test_df = pd.read_parquet(filepath)
                    print(f"✅ Successfully downloaded {filename} ({len(test_df):,} rows)")
                    downloaded_files.append(filepath)
                    break  # Found a good file, stop trying others
                except:
                    os.remove(filepath)  # Remove invalid file
                    print(f"❌ {filename} is not a valid parquet file")
            else:
                print(f"❌ {filename} not found (HTTP {response.status_code})")
                
        except Exception as e:
            print(f"❌ Error downloading {filename}: {e}")
    
    return downloaded_files

# Try downloading metadata from the correct source
print("🚀 Starting metadata download from correct source...")
downloaded_files = download_biotrove_metadata()

# If that fails, try direct download
if not downloaded_files:
    print("\n" + "="*60)
    print("🔄 Fallback: Trying direct file download...")
    downloaded_files = try_direct_download_from_hf()

if downloaded_files:
    print(f"\n🎉 Successfully obtained {len(downloaded_files)} metadata files!")
    
    # Verify the files we downloaded
    print("\n🔍 Analyzing downloaded metadata...")
    for file_path in downloaded_files:
        try:
            df = pd.read_parquet(file_path)
            print(f"✅ {os.path.basename(file_path)}: {len(df):,} rows, {len(df.columns)} columns")
            print(f"   🏷️ Columns: {list(df.columns)}")
            
            # Look for class/category information
            class_columns = [col for col in df.columns if 'class' in col.lower() or 'category' in col.lower()]
            if class_columns:
                for col in class_columns:
                    unique_classes = df[col].value_counts()
                    print(f"   📊 {col} distribution: {dict(unique_classes.head())}")
                    
                    # Check specifically for reptiles
                    if 'Reptilia' in unique_classes.index:
                        reptile_count = unique_classes['Reptilia']
                        print(f"   🦎 Found {reptile_count:,} Reptilia samples!")
            
            # Look for species information  
            if 'species' in df.columns:
                species_count = df['species'].nunique()
                print(f"   🦎 Unique species: {species_count:,}")
                
        except Exception as e:
            print(f"❌ Error reading {file_path}: {e}")
    
    print("\n✅ Metadata ready for BioTrove processing!")
    
else:
    print("\n❌ Could not download metadata from any source")
    print("\n🛠️ MANUAL DOWNLOAD INSTRUCTIONS:")
    print("1. Go to: https://huggingface.co/datasets/BGLab/BioTrove-Train") 
    print("2. Browse the files and download parquet files")
    print("3. Save them in the biotrove_metadata/ folder")
    print("4. Continue with processing steps")
    
print(f"\n💡 If no reptiles found, the dataset might be:")
print("   • Sorted by taxonomic class (Reptilia might be later)")
print("   • Very large (135M samples) - reptiles might be sparse")
print("   • Using different class names or structure")
print("   • Consider trying the comprehensive streaming approach in next cell")

In [ ]:
# COMPREHENSIVE METHOD: Strategic search through BioTrove-Train dataset
print("🔄 COMPREHENSIVE METHOD: Strategic search for Reptilia in BioTrove-Train")
print("💡 This uses skip patterns to sample different parts of the 135M dataset")

def extract_reptilia_with_skip_strategy():
    """Extract Reptilia samples using skip strategy to sample the entire dataset"""
    try:
        from datasets import load_dataset
        
        print("📡 Loading complete BGLab/BioTrove-Train dataset...")
        
        # Try different loading approaches
        try:
            # Use streaming for large dataset
            dataset = load_dataset("BGLab/BioTrove-Train", streaming=True)
            is_streaming = True
        except:
            try:
                # If streaming fails, try regular load
                dataset = load_dataset("BGLab/BioTrove-Train")
                is_streaming = False
            except:
                raise Exception("Could not load dataset in any mode")
        
        print(f"✅ Dataset loaded ({'streaming' if is_streaming else 'full'} mode)")
        
        # Get the main data split
        if 'train' in dataset:
            data_split = dataset['train']
        else:
            data_split = next(iter(dataset.values()))
        
        reptilia_samples = []
        total_processed = 0
        all_classes_found = set()
        
        # Strategy: Sample different parts of the dataset
        print(f"🔍 Using strategic sampling to find Reptilia in 135M samples...")
        print(f"💡 Will check samples at different intervals to cover the entire dataset")
        
        from tqdm.auto import tqdm
        pbar = tqdm(desc="Strategic sampling", unit=" samples")
        
        # Skip pattern: check every Nth sample to cover more ground
        skip_patterns = [1, 50, 500, 5000]  # Start dense, then sparse
        max_samples_per_pattern = 50000
        
        for skip_size in skip_patterns:
            print(f"\n🎯 Checking every {skip_size} samples...")
            pattern_samples = 0
            pattern_reptiles_found = 0
            
            sample_iter = iter(data_split)
            
            try:
                while pattern_samples < max_samples_per_pattern:
                    # Get the next item
                    for _ in range(skip_size):
                        try:
                            item = next(sample_iter)
                            total_processed += 1
                        except StopIteration:
                            print(f"📄 Reached end of dataset at {total_processed:,} samples")
                            break
                    else:
                        # Process this item
                        if isinstance(item, dict):
                            # Check multiple possible class/category fields
                            item_class = None
                            for field in ['class', 'category', 'taxonomic_class', 'tax_class']:
                                if field in item:
                                    item_class = str(item[field]).strip()
                                    all_classes_found.add(item_class)
                                    break
                            
                            # Check if this is a Reptilia sample
                            if item_class and (item_class == 'Reptilia' or 'reptil' in item_class.lower()):
                                reptilia_samples.append(item)
                                pattern_reptiles_found += 1
                                
                                if len(reptilia_samples) <= 10:
                                    species = item.get('species', item.get('scientificName', 'Unknown'))
                                    print(f"🦎 Found reptile #{len(reptilia_samples)}: {species}")
                                
                                pbar.set_description(f"Found {len(reptilia_samples)} Reptilia!")
                        
                        pattern_samples += 1
                        if pattern_samples % 1000 == 0:
                            pbar.update(1000)
                        
                        # Stop if we found enough reptiles
                        if len(reptilia_samples) >= 2000:
                            print(f"\n🎉 Found {len(reptilia_samples)} reptiles! Stopping search.")
                            break
                
                print(f"   Pattern results: {pattern_reptiles_found} reptiles from {pattern_samples:,} samples")
                
                # If we found reptiles, no need for sparser patterns
                if len(reptilia_samples) > 0:
                    break
                    
            except StopIteration:
                print(f"   Reached dataset end during pattern {skip_size}")
                break
        
        pbar.close()
        
        print(f"\n📊 SEARCH SUMMARY:")
        print(f"   Total samples processed: {total_processed:,}")
        print(f"   Classes found: {sorted(list(all_classes_found))}")
        print(f"   Reptilia samples found: {len(reptilia_samples)}")
        
        if reptilia_samples:
            # Save reptilia samples as parquet chunks
            chunk_size = 250
            saved_files = []
            
            print(f"\n💾 Saving {len(reptilia_samples)} Reptilia samples in chunks...")
            
            for i in range(0, len(reptilia_samples), chunk_size):
                chunk_data = reptilia_samples[i:i+chunk_size]
                chunk_df = pd.DataFrame(chunk_data)
                
                chunk_filename = f"biotrove_metadata/biotrove_reptilia_chunk_{i//chunk_size}.parquet"
                chunk_df.to_parquet(chunk_filename, index=False)
                saved_files.append(chunk_filename)
                
                print(f"✅ Chunk {i//chunk_size}: {len(chunk_data)} samples → {chunk_filename}")
            
            print(f"\n🎉 SUCCESS! Extracted {len(reptilia_samples)} Reptilia samples")
            print(f"📊 Processed {total_processed:,} total records with strategic sampling")
            print(f"💾 Created {len(saved_files)} metadata chunks")
            
            # Show sample species information
            all_reptilia_df = pd.DataFrame(reptilia_samples)
            if 'species' in all_reptilia_df.columns:
                species_counts = all_reptilia_df['species'].value_counts()
                print(f"🦎 Found {len(species_counts)} unique reptile species")
                print(f"   Top 5: {dict(species_counts.head())}")
            elif 'scientificName' in all_reptilia_df.columns:
                species_counts = all_reptilia_df['scientificName'].value_counts()
                print(f"🦎 Found {len(species_counts)} unique reptile species")
                print(f"   Top 5: {dict(species_counts.head())}")
            
            return saved_files
            
        else:
            print(f"\n❌ No Reptilia samples found even with strategic sampling")
            print(f"💡 The dataset might:")
            print(f"   • Not contain Reptilia class")
            print(f"   • Use different taxonomic naming")
            print(f"   • Have reptiles under a different field")
            
            # Save a larger sample to analyze structure
            print("💾 Creating larger analysis sample...")
            sample_data = []
            data_split_fresh = dataset['train'] if 'train' in dataset else next(iter(dataset.values()))
            
            for i, item in enumerate(data_split_fresh):
                sample_data.append(item)
                if i >= 2000:  # Get more samples for analysis
                    break
            
            if sample_data:
                sample_df = pd.DataFrame(sample_data)
                sample_file = "biotrove_metadata/biotrove_large_sample.parquet"
                sample_df.to_parquet(sample_file, index=False)
                
                print(f"📊 Large sample saved to: {sample_file}")
                print(f"   Columns: {list(sample_df.columns)}")
                print(f"   Shape: {sample_df.shape}")
                
                # Detailed class analysis
                for col in sample_df.columns:
                    if any(keyword in col.lower() for keyword in ['class', 'category', 'tax', 'phylum']):
                        unique_vals = sample_df[col].value_counts()
                        print(f"   {col} distribution: {dict(unique_vals.head(10))}")
                
                return [sample_file]
            
            return []
            
    except Exception as e:
        print(f"❌ Error processing BioTrove-Train dataset: {e}")
        print("\n💡 Possible issues:")
        print("   - Dataset access problems")
        print("   - Network connectivity issues") 
        print("   - Dataset structure different than expected")
        return []

# If previous methods didn't work, try strategic search
if not downloaded_files or not any('reptilia' in f.lower() for f in downloaded_files):
    print("\n" + "="*60)
    print("🔄 Trying strategic search through BioTrove-Train dataset...")
    new_files = extract_reptilia_with_skip_strategy()
    
    # Add any new files found
    if new_files:
        downloaded_files.extend(new_files)

print(f"\n🏁 FINAL RESULT: {len(downloaded_files)} metadata files available")
if downloaded_files:
    # Check if any files actually contain reptiles
    reptile_files = [f for f in downloaded_files if 'reptilia' in f.lower()]
    if reptile_files:
        print(f"✅ Found {len(reptile_files)} files with Reptilia data!")
        print("📝 Ready to proceed with BioTrove processing pipeline!")
    else:
        print("⚠️ Files available but may not contain Reptilia data")
        print("📊 Files contain sample data for structure analysis")
else:
    print("❌ Could not obtain any metadata files")
    print("💡 Consider manual download from https://huggingface.co/datasets/BGLab/BioTrove-Train")

In [ ]:
# Create configuration for BioTrove processing focused on Reptilia
config = {
    "metadata_processor_info": {
        "source_folder": "biotrove_metadata",
        "destination_folder": "biotrove_processed", 
        "categories": ["Reptilia"]  # Focus only on reptile data
    },
    "metadata_filter_and_shuffle_info": {
        "species_count_data": "biotrove_processed/species_category_counts.csv",
        "directory": "biotrove_metadata",
        "rare_threshold": 5,      # Minimum samples per species
        "cap_threshold": 500,     # Maximum samples per species for balanced dataset
        "part_size": 100,         # Samples per processing chunk
        "rare_dir": "biotrove_processed/rare_cases",
        "cap_filtered_dir_train": "biotrove_processed/cap_filtered_train", 
        "capped_dir": "biotrove_processed/capped_cases",
        "merged_dir": "biotrove_processed/merged_cases",
        "files_per_chunk": 3,
        "random_seed": 42
    },
    "image_download_info": {
        "input_folder": "biotrove_processed/cap_filtered_train",
        "output_folder": "reptile_images", 
        "start_index": 0,
        "end_index": 10,          # Limit for demo - increase for full processing
        "concurrent_downloads": 50  # Adjust based on your internet connection
    },
    "img_text_gen_info": {
        "metadata": "biotrove_processed/cap_filtered_train",
        "img_folder": "reptile_images",
        "output_base_folder": "reptile_dataset"
    }
}

# Save configuration file
with open('biotrove_reptilia_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("⚙️ Configuration created for Reptilia processing:")
print("   🎯 Target: Reptilia class only")  
print("   📊 Rare threshold: ≥5 samples per species")
print("   ⚖️ Cap threshold: ≤500 samples per species")
print("   💾 Config saved to: biotrove_reptilia_config.json")

In [ ]:
# Step 1: Process metadata files to extract Reptilia samples
print("🔍 Step 1: Processing metadata to extract Reptilia samples...")

# First, let's check what metadata we actually have
metadata_dir = "biotrove_metadata"
if os.path.exists(metadata_dir):
    metadata_files = [f for f in os.listdir(metadata_dir) if f.endswith('.parquet')]
    print(f"📂 Found {len(metadata_files)} metadata files: {metadata_files}")
    
    if metadata_files:
        # Check the content of the first file
        sample_file = os.path.join(metadata_dir, metadata_files[0])
        sample_df = pd.read_parquet(sample_file)
        
        print(f"\n🔍 Examining metadata structure:")
        print(f"   📏 Shape: {sample_df.shape}")
        print(f"   🏷️ Columns: {list(sample_df.columns)}")
        
        # Check for class/category information
        class_columns = [col for col in sample_df.columns if 'class' in col.lower() or 'category' in col.lower()]
        if class_columns:
            for col in class_columns:
                unique_vals = sample_df[col].unique()
                print(f"   📊 {col} values: {unique_vals[:10]}")
                
                # Check if we have reptiles
                reptile_mask = sample_df[col].astype(str).str.contains('reptil', case=False, na=False)
                reptile_count = reptile_mask.sum()
                if reptile_count > 0:
                    print(f"   🦎 Found {reptile_count} Reptilia samples!")
                else:
                    print(f"   ⚠️ No Reptilia samples found in {col}")
        
        # Check if this looks like actual BioTrove data or just a sample
        if sample_df.shape[0] <= 1000 and 'sample_structure' in metadata_files[0]:
            print(f"\n⚠️ WARNING: This appears to be a sample/structure file, not actual reptile metadata")
            print(f"💡 You need to download actual Reptilia data from BGLab/BioTrove-Train")
            print(f"📝 Suggestion: Re-run the metadata download cells (7-8) to get real reptile data")
        
        has_reptiles = any(
            sample_df[col].astype(str).str.contains('reptil', case=False, na=False).any() 
            for col in class_columns
        )
        
        if not has_reptiles:
            print(f"\n❌ No Reptilia data available for processing")
            print(f"🛠️ SOLUTIONS:")
            print(f"   1. Re-run cells 7-8 to download actual reptile metadata")
            print(f"   2. Check internet connection and dataset access")
            print(f"   3. Consider using the streaming approach in cell 8")

try:
    # Only proceed if we have metadata files and reptiles
    if metadata_files and has_reptiles:
        # Load configuration
        config = load_config('biotrove_reptilia_config.json')
        
        # Initialize MetadataProcessor
        params = config.get('metadata_processor_info', {})
        mp = MetadataProcessor(**params)
        
        print("\n📊 Processing metadata files with BioTrove tools...")
        mp.process_all_files()
        
        print("✅ Metadata processing complete!")
        
        # Check results
        species_counts_file = "biotrove_processed/species_category_counts.csv"
        if os.path.exists(species_counts_file):
            species_counts = pd.read_csv(species_counts_file)
            print(f"📈 Found {len(species_counts)} species records")
            
            # Show reptile species if any found
            if 'category' in species_counts.columns:
                reptile_species = species_counts[species_counts['category'] == 'Reptilia']
            else:
                # Try alternative column names
                alt_cols = [col for col in species_counts.columns if 'class' in col.lower()]
                if alt_cols:
                    col_name = alt_cols[0]
                    reptile_species = species_counts[species_counts[col_name] == 'Reptilia']
                else:
                    reptile_species = pd.DataFrame()
            
            if len(reptile_species) > 0:
                print(f"🦎 Reptilia species found: {len(reptile_species)}")
                print("🏆 Top 5 Reptilia species by count:")
                
                # Ensure count column is numeric
                count_col = 'count'
                if count_col in reptile_species.columns:
                    reptile_species[count_col] = pd.to_numeric(reptile_species[count_col], errors='coerce')
                    top_species = reptile_species.nlargest(5, count_col)
                    for _, row in top_species.iterrows():
                        species_name = row.get('species', 'Unknown')
                        count_val = row.get(count_col, 0)
                        print(f"   {species_name}: {count_val} samples")
                else:
                    print(f"   ⚠️ Count column not found in results")
            else:
                print("⚠️ No Reptilia samples found in the processed chunks")
                print("💡 This confirms we need actual reptile metadata, not sample data")
        else:
            print(f"❌ Species count file not created: {species_counts_file}")
            print("💡 The metadata processing failed - likely due to wrong data format")
    else:
        print("❌ No metadata files found to process")
        
except Exception as e:
    print(f"❌ Error in metadata processing: {str(e)}")
    print("💡 Common issues:")
    print("   • Wrong data format (need actual BioTrove metadata, not sample files)")
    print("   • Missing or incompatible columns")
    print("   • Data type mismatches")
    print("   • Ensure arbor_process is properly installed")

print("\n" + "="*60)

# Step 2: Filter and shuffle the data
print("🔄 Step 2: Filtering and shuffling Reptilia data...")

try:
    # Check if the species count file exists first
    species_counts_file = "biotrove_processed/species_category_counts.csv"
    if not os.path.exists(species_counts_file):
        print("❌ Cannot proceed: species_category_counts.csv not found")
        print("💡 Step 1 must complete successfully before running Step 2")
        print("🛠️ Please fix the metadata issues in Step 1 first")
    else:
        config = load_config('biotrove_reptilia_config.json')
        params = config.get('metadata_filter_and_shuffle_info', {})
        gen_shuffled_chunks = GenShuffledChunks(**params)
        
        print("⚖️ Applying rare/cap thresholds and shuffling...")
        gen_shuffled_chunks.process_files()
        
        print("✅ Data filtering and shuffling complete!")
        
        # Check filtered results
        cap_filtered_dir = params.get('cap_filtered_dir_train', 'biotrove_processed/cap_filtered_train')
        if os.path.exists(cap_filtered_dir):
            filtered_files = [f for f in os.listdir(cap_filtered_dir) if f.endswith('.parquet')]
            print(f"📂 Created {len(filtered_files)} filtered data chunks")
            
            if filtered_files:
                # Sample one file to see content
                sample_file = os.path.join(cap_filtered_dir, filtered_files[0])
                sample_df = pd.read_parquet(sample_file)
                print(f"📊 Sample chunk contains {len(sample_df)} records")
                if len(sample_df) > 0:
                    if 'species' in sample_df.columns:
                        print(f"🏷️ Sample species: {sample_df['species'].unique()[:3]}")
                    else:
                        print(f"🏷️ Sample columns: {list(sample_df.columns)}")
        else:
            print("⚠️ Filtered data directory not created")
    
except Exception as e:
    print(f"❌ Error in filtering/shuffling: {str(e)}")
    print("💡 Ensure Step 1 completed successfully")

print("\n" + "="*60)
print("✅ Metadata processing pipeline complete!")
print("\n📝 NEXT STEPS:")
if os.path.exists("biotrove_processed/cap_filtered_train"):
    print("   ✅ Ready to run image download step!")
else:
    print("   ❌ Need to get actual Reptilia metadata first")
    print("   🔄 Re-run cells 7-8 to download real reptile data from BGLab/BioTrove-Train")
    print("   📊 Current data appears to be sample/structure data only")

In [ ]:
# Analyze the processed reptilia data
def analyze_processed_reptilia_data(processed_dir="biotrove_processed"):
    """Analyze the processed reptilia dataset"""
    
    print("🔍 Analyzing processed Reptilia dataset...")
    
    # Check species counts
    species_file = os.path.join(processed_dir, "species_category_counts.csv")
    if os.path.exists(species_file):
        df = pd.read_csv(species_file)
        reptile_data = df[df['category'] == 'Reptilia']
        
        if len(reptile_data) > 0:
            print(f"\n📊 REPTILIA DATASET SUMMARY")
            print("="*50)
            print(f"Total reptile species: {len(reptile_data)}")
            print(f"Total reptile samples: {reptile_data['count'].sum():,}")
            print(f"Average samples per species: {reptile_data['count'].mean():.1f}")
            print(f"Median samples per species: {reptile_data['count'].median():.1f}")
            
            # Top families analysis
            if 'family' in reptile_data.columns:
                family_counts = reptile_data.groupby('family')['count'].sum().sort_values(ascending=False)
                print(f"\n🏆 Top 5 Reptile Families:")
                for family, count in family_counts.head().items():
                    print(f"   {family}: {count:,} samples")
            
            # Species distribution
            print(f"\n📈 Species Distribution:")
            print(f"   Species with >100 samples: {(reptile_data['count'] > 100).sum()}")
            print(f"   Species with 50-100 samples: {((reptile_data['count'] >= 50) & (reptile_data['count'] <= 100)).sum()}")
            print(f"   Species with 10-49 samples: {((reptile_data['count'] >= 10) & (reptile_data['count'] < 50)).sum()}")
            print(f"   Species with <10 samples: {(reptile_data['count'] < 10).sum()}")
            
            return reptile_data
        else:
            print("❌ No Reptilia data found in processed results")
    else:
        print(f"❌ Species count file not found: {species_file}")
    
    return None

# Run analysis
reptile_analysis = analyze_processed_reptilia_data()

# Check filtered data chunks
cap_filtered_dir = "biotrove_processed/cap_filtered_train"
if os.path.exists(cap_filtered_dir):
    filtered_files = [f for f in os.listdir(cap_filtered_dir) if f.endswith('.parquet')]
    print(f"\n📂 Filtered data chunks available: {len(filtered_files)}")
    
    if filtered_files:
        print("📋 Ready for image download!")
    else:
        print("⚠️ No filtered chunks found - may need more metadata or different thresholds")
else:
    print("⚠️ Filtered data directory not found")

In [ ]:
# Step 3: Download reptilia images using official BioTrove tools
print("📥 Step 3: Downloading Reptilia images...")

async def download_reptilia_images():
    """Download reptile images using the official BioTrove image downloader"""
    try:
        # Load configuration
        config = load_config('biotrove_reptilia_config.json')
        params = config.get('image_download_info', {})
        
        # Initialize image downloader
        gi = GetImages(**params)
        
        print(f"🖼️ Starting image download...")
        print(f"   📂 Input folder: {params['input_folder']}")
        print(f"   💾 Output folder: {params['output_folder']}")
        print(f"   📊 Processing chunks {params['start_index']} to {params['end_index']}")
        print(f"   🔄 Concurrent downloads: {params['concurrent_downloads']}")
        print()
        
        # Create output directory
        os.makedirs(params['output_folder'], exist_ok=True)
        
        # Download images
        await gi.download_images()
        
        print("✅ Image download complete!")
        
        # Check results
        output_folder = params['output_folder']
        if os.path.exists(output_folder):
            # Count downloaded images
            total_images = 0
            species_folders = []
            
            for item in os.listdir(output_folder):
                item_path = os.path.join(output_folder, item)
                if os.path.isdir(item_path):
                    species_folders.append(item)
                    images_in_folder = len([f for f in os.listdir(item_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    total_images += images_in_folder
            
            print(f"\n📊 Download Results:")
            print(f"   🖼️ Total images downloaded: {total_images}")
            print(f"   🦎 Species folders created: {len(species_folders)}")
            
            if species_folders:
                print(f"   📂 Sample species folders: {species_folders[:5]}")
        
    except Exception as e:
        print(f"❌ Error downloading images: {str(e)}")
        print("💡 Check that filtered data exists and internet connection is stable")

# Note: For Jupyter, we need to handle async differently
print("⏳ Starting image download process...")
print("💡 This may take several minutes depending on the number of images...")

# Check if we have filtered data to download
cap_filtered_dir = "biotrove_processed/cap_filtered_train" 
if os.path.exists(cap_filtered_dir):
    filtered_files = [f for f in os.listdir(cap_filtered_dir) if f.endswith('.parquet')]
    if filtered_files:
        print(f"📂 Found {len(filtered_files)} filtered data chunks to process")
        
        # Run the async download
        try:
            # For Jupyter compatibility
            import nest_asyncio
            nest_asyncio.apply()
            await download_reptilia_images()
        except ImportError:
            print("⚠️ nest_asyncio not available. Installing...")
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
            import nest_asyncio
            nest_asyncio.apply()
            await download_reptilia_images()
    else:
        print("❌ No filtered data chunks found. Run the metadata processing steps first.")
else:
    print("❌ Filtered data directory not found. Run the metadata processing steps first.")

In [ ]:
# Step 4: Generate text labels and create final dataset
print("📝 Step 4: Generating text labels for Reptilia images...")

def generate_reptilia_text_labels():
    """Generate text labels for downloaded reptile images"""
    try:
        # Load configuration
        config = load_config('biotrove_reptilia_config.json')
        params = config.get('img_text_gen_info', {})
        
        # Initialize text generator
        textgen = GenImgTxtPair(**params)
        
        print("🏷️ Creating image-text pairs...")
        print(f"   📂 Metadata source: {params['metadata']}")
        print(f"   🖼️ Image folder: {params['img_folder']}")
        print(f"   💾 Output folder: {params['output_base_folder']}")
        
        # Create output directory
        os.makedirs(params['output_base_folder'], exist_ok=True)
        
        # Generate image-text pairs
        textgen.create_image_text_pairs()
        
        print("✅ Text label generation complete!")
        
        # Check results
        output_folder = params['output_base_folder']
        if os.path.exists(output_folder):
            # Count generated files
            tar_files = [f for f in os.listdir(output_folder) if f.endswith('.tar')]
            txt_files = []
            json_files = []
            
            # Check subfolders for text files
            for item in os.listdir(output_folder):
                item_path = os.path.join(output_folder, item)
                if os.path.isdir(item_path):
                    txt_files.extend([f for f in os.listdir(item_path) if f.endswith('.txt')])
                    json_files.extend([f for f in os.listdir(item_path) if f.endswith('.json')])
            
            print(f"\n📊 Generated Dataset:")
            print(f"   📦 TAR files: {len(tar_files)}")
            print(f"   📝 Text files: {len(txt_files)}")
            print(f"   📄 JSON files: {len(json_files)}")
            
            # Show sample text labels if available
            if txt_files:
                sample_txt_path = None
                for item in os.listdir(output_folder):
                    item_path = os.path.join(output_folder, item)
                    if os.path.isdir(item_path):
                        txt_files_in_dir = [f for f in os.listdir(item_path) if f.endswith('.txt')]
                        if txt_files_in_dir:
                            sample_txt_path = os.path.join(item_path, txt_files_in_dir[0])
                            break
                
                if sample_txt_path and os.path.exists(sample_txt_path):
                    print(f"\n📝 Sample text label:")
                    with open(sample_txt_path, 'r') as f:
                        sample_text = f.read().strip()
                        print(f"   '{sample_text[:100]}...'")
        
        return True
        
    except Exception as e:
        print(f"❌ Error generating text labels: {str(e)}")
        return False

# Generate text labels
if generate_reptilia_text_labels():
    print("\n🎉 BIOTROVE REPTILIA PROCESSING COMPLETE!")
    print("="*60)
    print("✅ Your reptile dataset is ready for computer vision tasks!")
    print("\n📁 Dataset Structure:")
    print("   📂 biotrove_metadata/          - Raw metadata chunks")
    print("   📂 biotrove_processed/         - Processed species data") 
    print("   📂 reptile_images/             - Downloaded reptile images")
    print("   📂 reptile_dataset/            - Final image-text pairs")
    print("\n🔬 Ready for:")
    print("   • CNN training and fine-tuning")
    print("   • Vision Transformer experiments") 
    print("   • YOLOv10 object detection")
    print("   • BioTrove-CLIP model evaluation")
    print("   • Conservation and biodiversity applications")
else:
    print("⚠️ Text label generation failed. Check previous steps.")

# Summary statistics
def show_final_dataset_summary():
    """Show final summary of the created dataset"""
    print(f"\n📊 FINAL DATASET SUMMARY")
    print("="*40)
    
    # Count images
    img_folder = "reptile_images"
    total_images = 0
    species_count = 0
    
    if os.path.exists(img_folder):
        for item in os.listdir(img_folder):
            item_path = os.path.join(img_folder, item)
            if os.path.isdir(item_path):
                species_count += 1
                images = [f for f in os.listdir(item_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                total_images += len(images)
    
    # Count processed species
    species_file = "biotrove_processed/species_category_counts.csv"
    total_species_in_metadata = 0
    if os.path.exists(species_file):
        df = pd.read_csv(species_file)
        reptile_data = df[df['category'] == 'Reptilia']
        total_species_in_metadata = len(reptile_data)
    
    print(f"🦎 Reptile species in metadata: {total_species_in_metadata}")
    print(f"📂 Species with downloaded images: {species_count}")
    print(f"🖼️ Total reptile images: {total_images}")
    
    # Show dataset is ready for herpeton project goals
    if total_images > 0:
        print(f"\n🎯 READY FOR HERPETON PROJECT:")
        print(f"   ✅ Species classification")
        print(f"   ✅ Family-level taxonomy")
        print(f"   ✅ Transfer learning experiments")
        print(f"   ✅ Conservation applications")

show_final_dataset_summary()

In [ ]:
# Explore and visualize the reptilia dataset
def explore_reptile_dataset():
    """Explore the processed reptile dataset"""
    
    print("🔍 Exploring Reptile Dataset...")
    
    # Load species data
    species_file = "biotrove_processed/species_category_counts.csv"
    if not os.path.exists(species_file):
        print("❌ Species data not found. Run processing steps first.")
        return
    
    df = pd.read_csv(species_file)
    reptile_data = df[df['category'] == 'Reptilia'].copy()
    
    if len(reptile_data) == 0:
        print("❌ No reptile data found.")
        return
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Top species
    top_species = reptile_data.nlargest(10, 'count')
    top_species.plot(x='species', y='count', kind='bar', ax=axes[0,0], color='skyblue')
    axes[0,0].set_title('Top 10 Reptile Species by Sample Count')
    axes[0,0].set_xlabel('Species')
    axes[0,0].set_ylabel('Sample Count')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Family analysis if available
    if 'family' in reptile_data.columns:
        family_counts = reptile_data.groupby('family')['count'].sum().sort_values(ascending=False)
        family_counts.head(8).plot(kind='bar', ax=axes[0,1], color='lightcoral')
        axes[0,1].set_title('Top 8 Reptile Families')
        axes[0,1].set_xlabel('Family')
        axes[0,1].set_ylabel('Total Samples')
        axes[0,1].tick_params(axis='x', rotation=45)
    
    # Sample distribution
    reptile_data['count'].hist(bins=20, ax=axes[1,0], color='lightgreen', alpha=0.7)
    axes[1,0].set_title('Distribution of Samples per Species')
    axes[1,0].set_xlabel('Samples per Species')
    axes[1,0].set_ylabel('Number of Species')
    axes[1,0].set_yscale('log')
    
    # Cumulative coverage
    sorted_counts = reptile_data['count'].sort_values(ascending=False)
    cumsum_pct = (sorted_counts.cumsum() / sorted_counts.sum() * 100)
    axes[1,1].plot(range(len(cumsum_pct)), cumsum_pct, color='purple')
    axes[1,1].set_title('Cumulative Species Coverage')
    axes[1,1].set_xlabel('Species Rank')
    axes[1,1].set_ylabel('Cumulative % of Samples')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return reptile_data

# Run exploration
reptile_stats = explore_reptile_dataset()

In [ ]:
def create_full_reptilia_dataset(save_path="full_reptilia_dataset.csv", batch_size=10000):
    """
    Process the entire BioTrove dataset to extract all Reptilia samples.
    This function is designed to run for a long time and save progress incrementally.
    
    Args:
        save_path (str): Path to save the complete dataset
        batch_size (int): Number of samples to process before saving progress
    """
    print("🔥 Starting full Reptilia dataset creation...")
    print("⚠️  WARNING: This will take several hours to complete!")
    print("💡 You can stop anytime with Ctrl+C and resume later")
    
    # Check if we're resuming from a previous run
    start_idx = 0
    existing_data = []
    
    if os.path.exists(save_path):
        existing_df = pd.read_csv(save_path)
        existing_data = existing_df.to_dict('records')
        print(f"📂 Resuming from existing file with {len(existing_data):,} samples")
    
    # Load dataset in streaming mode
    dataset = load_dataset("BGLab/BioTrove-Train", streaming=True)
    train_data = dataset['train']
    
    all_reptilia_data = existing_data.copy()
    
    try:
        reptilia_count = len(existing_data)
        total_processed = 0
        
        print("\n🚀 Processing dataset...")
        pbar = tqdm(desc="Searching for reptiles", unit=" samples")
        
        for item in train_data:
            total_processed += 1
            
            # Check if this is a reptilia sample
            if item.get('class') == 'Reptilia':
                all_reptilia_data.append(item)
                reptilia_count += 1
            
            # Update progress every 1000 samples
            if total_processed % 1000 == 0:
                pbar.set_description(f"Processed {total_processed:,} | Found {reptilia_count:,} reptiles")
                pbar.update(1000)
            
            # Save progress every batch_size samples
            if total_processed % batch_size == 0 and reptilia_count > len(existing_data):
                temp_df = pd.DataFrame(all_reptilia_data)
                temp_df.to_csv(save_path, index=False)
                print(f"\n💾 Progress saved: {reptilia_count:,} reptiles from {total_processed:,} samples")
        
    except KeyboardInterrupt:
        print(f"\n⏹️ Processing interrupted by user")
        
    finally:
        pbar.close()
        
        # Final save
        if all_reptilia_data:
            final_df = pd.DataFrame(all_reptilia_data)
            final_df.to_csv(save_path, index=False)
            
            print(f"\n🎉 FINAL RESULTS:")
            print(f"📊 Total samples processed: {total_processed:,}")
            print(f"🦎 Total reptiles found: {len(all_reptilia_data):,}")
            print(f"🔢 Unique species: {final_df['species'].nunique():,}")
            print(f"💾 Saved to: {save_path}")
            
            return final_df
        else:
            print("❌ No data collected")
            return pd.DataFrame()

print("⚡ Full dataset processing function ready!")
print("💡 Call create_full_reptilia_dataset() to start processing the complete dataset")

In [ ]:
# Example: Get species for specific families
def get_species_by_family(df, family_name):
    """
    Get all species from a specific reptile family.
    
    Args:
        df (pd.DataFrame): Reptilia dataset
        family_name (str): Name of the family (e.g., 'Viperidae', 'Gekkonidae')
    
    Returns:
        pd.DataFrame: Species information for the family
    """
    family_data = df[df['family'] == family_name]
    
    if len(family_data) == 0:
        print(f"❌ No species found in family: {family_name}")
        return pd.DataFrame()
    
    species_summary = family_data.groupby(['species', 'common_name']).size().reset_index()
    species_summary.columns = ['Species', 'Common Name', 'Sample Count']
    species_summary = species_summary.sort_values('Sample Count', ascending=False)
    
    print(f"🐍 Family: {family_name}")
    print(f"📊 Total species: {len(species_summary)}")
    print(f"📊 Total samples: {len(family_data):,}")
    
    return species_summary

# Example: Create a balanced dataset
def create_balanced_subset(df, min_samples=10, max_samples=100):
    """
    Create a balanced subset with similar number of samples per species.
    
    Args:
        df (pd.DataFrame): Full reptilia dataset
        min_samples (int): Minimum samples required per species
        max_samples (int): Maximum samples to keep per species
    
    Returns:
        pd.DataFrame: Balanced subset
    """
    species_counts = df['species'].value_counts()
    
    # Filter species with sufficient samples
    valid_species = species_counts[species_counts >= min_samples].index
    print(f"📊 Species with ≥{min_samples} samples: {len(valid_species)}/{len(species_counts)}")
    
    # Sample from each valid species
    balanced_data = []
    for species in valid_species:
        species_data = df[df['species'] == species]
        sample_size = min(max_samples, len(species_data))
        sampled = species_data.sample(n=sample_size, random_state=42)
        balanced_data.append(sampled)
    
    balanced_df = pd.concat(balanced_data, ignore_index=True)
    
    print(f"✅ Created balanced dataset:")
    print(f"   📊 Total samples: {len(balanced_df):,}")
    print(f"   🦎 Species included: {balanced_df['species'].nunique()}")
    print(f"   📈 Samples per species: {min_samples}-{max_samples}")
    
    return balanced_df

print("✅ Utility functions created!")
print("\n📚 Available functions:")
print("  • get_species_by_family(df, family_name)")
print("  • create_balanced_subset(df, min_samples, max_samples)")
print("  • show_species_samples(df, species_name, max_images)")
print("  • create_full_reptilia_dataset() - for complete dataset extraction")